# Text generation with an RNN

### libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path = "./dataset/shahnameh.txt"

### Read the data

In [3]:
f = open(path, 'rb')
text = f.read().decode(encoding='utf-8')

print(len(text)) # length of text

2653849


In [6]:
print(text[:250])

|به نام خداوند جان و خرد
|کزین برتر اندیشه برنگذرد
|خداوند نام و خداوند جای
|خداوند روزی ده رهنمای
|خداوند کیوان و گردان سپهر
|فروزنده ماه و ناهید و مهر
|ز نام و نشان و گمان برترست
|نگارندهٔ بر شده پیکرست
|به بینندگان آفریننده را
|نبینی مرنجان دو بین


In [16]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

vocab

48 unique characters


['\n',
 ' ',
 '(',
 ')',
 '|',
 '«',
 '»',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

## Process the text

### Vectorize the text

In [13]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [15]:
char2idx

{'\n': 0,
 ' ': 1,
 '(': 2,
 ')': 3,
 '|': 4,
 '«': 5,
 '»': 6,
 '،': 7,
 '؟': 8,
 'ء': 9,
 'آ': 10,
 'أ': 11,
 'ؤ': 12,
 'ئ': 13,
 'ا': 14,
 'ب': 15,
 'ت': 16,
 'ث': 17,
 'ج': 18,
 'ح': 19,
 'خ': 20,
 'د': 21,
 'ذ': 22,
 'ر': 23,
 'ز': 24,
 'س': 25,
 'ش': 26,
 'ص': 27,
 'ض': 28,
 'ط': 29,
 'ظ': 30,
 'ع': 31,
 'غ': 32,
 'ف': 33,
 'ق': 34,
 'ل': 35,
 'م': 36,
 'ن': 37,
 'ه': 38,
 'و': 39,
 'ٔ': 40,
 'پ': 41,
 'چ': 42,
 'ژ': 43,
 'ک': 44,
 'گ': 45,
 'ی': 46,
 '\u200c': 47}

In [14]:
text_as_int = np.array([char2idx[c] for c in text])
text_as_int.shape

(2653849,)

In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'|به نام خداون' ---- characters mapped to int ---- > [ 4 15 38  1 37 14 36  1 20 21 14 39 37]


### Create training examples and targets

In [10]:
# The maximum length sentence
seq_length = 100

# Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
    print(i.numpy(), end = ' = ')
    print(idx2char[i.numpy()])

4 = |
15 = ب
38 = ه
1 =  
37 = ن
14 = ا
36 = م
1 =  
20 = خ
21 = د
14 = ا
39 = و
37 = ن
21 = د
1 =  


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(3):
    print(repr(''.join(idx2char[item.numpy()])))
    print("\n****"*2)

'|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'

****
****
'داوند کیوان و گردان سپهر\n|فروزنده ماه و ناهید و مهر\n|ز نام و نشان و گمان برترست\n|نگارندهٔ بر شده پیکر'

****
****
'ست\n|به بینندگان آفریننده را\n|نبینی مرنجان دو بیننده را\n|نیابد بدو نیز اندیشه راه\n|که او برتر از نام و'

****
****


In [12]:
# For each sequence, duplicate and shift it to form the input and target text
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'
Target data: 'به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:10], target_example[:10])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 4 ('|')
  expected output: 15 ('ب')
Step    1
  input: 15 ('ب')
  expected output: 38 ('ه')
Step    2
  input: 38 ('ه')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 37 ('ن')
Step    4
  input: 37 ('ن')
  expected output: 14 ('ا')
Step    5
  input: 14 ('ا')
  expected output: 36 ('م')
Step    6
  input: 36 ('م')
  expected output: 1 (' ')
Step    7
  input: 1 (' ')
  expected output: 20 ('خ')
Step    8
  input: 20 ('خ')
  expected output: 21 ('د')
Step    9
  input: 21 ('د')
  expected output: 14 ('ا')


### Create training batches

In [15]:
BATCH_SIZE = 32

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int32, tf.int32)>

## Build The Model

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 25
# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(int(rnn_units/2),
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
model = build_model(
  vocab_size=len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
# try model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model.predict(input_example_batch)
    print(example_batch_predictions.shape, "#(batch_size, sequence_length, vocab_size)")

(32, 100, 48) #(batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 25)            1200      
_________________________________________________________________
lstm (LSTM)                  (32, None, 512)           1101824   
_________________________________________________________________
lstm_1 (LSTM)                (32, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (32, None, 48)            49200     
Total params: 7,447,776
Trainable params: 7,447,776
Non-trainable params: 0
_________________________________________________________________


In [21]:
# sample from the output distribution, to get actual character indices
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([ 0,  7, 39, 28,  2, 22, 23, 35,  6, 31, 20,  7,  5, 37, 19,  2, 21,
       12,  7, 11, 17, 38,  6, 11, 44, 18, 30,  4, 35, 47,  5,  6, 11, 19,
       40,  4, 33, 42, 21, 14, 35, 47, 19, 19, 36, 31, 18, 29, 15, 15,  6,
       33, 27, 24,  4, 23, 42, 23, 38,  1, 44,  0, 37,  1, 30, 21, 47, 45,
       15, 24, 36, 32, 24, 21, 11,  9, 27, 42, 17, 24, 28, 12,  6, 12, 32,
       45, 16,  9, 47, 31, 12, 45, 46, 15, 40,  7,  1, 13, 25, 31],
      dtype=int64)

In [23]:
# Decode these to see the text predicted
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'

Next Char Predictions: 
 '\n،وض(ذرل»عخ،«نح(دؤ،أثه»أکجظ|ل\u200c«»أحٔ|فچدال\u200cححمعجطبب»فصز|رچره ک\nن ظد\u200cگبزمغزدأءصچثزضؤ»ؤغگتء\u200cعؤگیبٔ، ئسع'


## Train the model

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [28]:
EPOCHS = 15
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
821/821 [==============================] - 56s 67ms/step - loss: 2.1872 - accuracy: 0.3713
Epoch 2/15
821/821 [==============================] - 56s 68ms/step - loss: 1.4672 - accuracy: 0.5621
Epoch 3/15
821/821 [==============================] - 56s 69ms/step - loss: 1.3327 - accuracy: 0.5974
Epoch 4/15
821/821 [==============================] - 57s 69ms/step - loss: 1.2634 - accuracy: 0.6165
Epoch 5/15
821/821 [==============================] - 57s 70ms/step - loss: 1.2105 - accuracy: 0.6311
Epoch 6/15
821/821 [==============================] - 57s 70ms/step - loss: 1.1636 - accuracy: 0.6442
Epoch 7/15
821/821 [==============================] - 58s 71ms/step - loss: 1.1204 - accuracy: 0.6564
Epoch 8/15
821/821 [==============================] - 58s 71ms/step - loss: 1.0809 - accuracy: 0.6678
Epoch 9/15
821/821 [==============================] - 58s 71ms/step - loss: 1.0444 - accuracy: 0.6781
Epoch 10/15
821/821 [==============================] - 58s 71ms/step - loss: 1.011

## Generate text

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

### The prediction loop

In [32]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []


    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [36]:
print(generate_text(model, start_string=u"گهی با رستم"))

گهی با رستم شیردل رستخیز
|به افراسیاب آن خوره نوشد چه گریان زرین نهید
|پراز برف چاچین و با دانیم
|یکی تیر زد بر کران تاج وتخت
|شما را ز عنبر بسی سرخ و زشت
|کجا آن بزرگان چو خرم بهار
|گهرها ز گیتی مبندی بود
|چنویتر
|ز پروردهٔ شاه نوشین روان
|چو هشتادسان بوده باشد نگاه
|به پیش پدر سرخ گردنده هست
|کجا توشهٔ شهریاری بود
|بدو ماه و خورشید و فریادرس
|مهندست مردی ز بیم گزند
|که مه سالخورشخونشی و بانگ بود
|بدین آرزو رای و فرمان تو راست
|بود روز دستان پس شهریار
|همان پیشکابا برو گوی تهر
|ع
|همی برتو خواهی که او را گزید
|بناگاه شد سوی آن رزمگاه
|شد آن کشته آن کوه و هم دشمنست
|کزو یادگیری همان سرزنش
|به کاری نجستست با من به جنگ
|جهان بی‌هنر وسپی خیره گشت
|که کار آمدمانان خورش
|ز کافور مغز تو را کن بساخت
|ز دیهیم شاهی و تخت و کلاه
|کس اندر جهان این شود گوشتند
|ببست و بیاراست و آمد به کار
|ششم تیره گر زند و استا مگوی
|چنین داد پاسخ ورا کی قرا
|بیارد کس او را نشاید نشان
|گه آمد کنی خویشتن در جهان
|ز بالا ببارید بر چارباد
|گر از پوشش از بوی و رگشان رسید
|بجستند تازان به بانگ خروس
|ز بدخواه برخواندم
|